In [1]:
from dotenv import load_dotenv
from langchain_community.vectorstores import SupabaseVectorStore
from supabase.client import create_client

from langchain_ollama import OllamaEmbeddings

from db_schema_builder import SchemaDocumentBuilder
from db_schema_extractor import DatabaseSchemaExtractor

from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.utilities import SQLDatabase
from langchain.chains import RetrievalQAWithSourcesChain, RetrievalQA
from supabase import create_client, Client
from langchain_openai import ChatOpenAI
from os import getenv
from pprint import pprint

import json

load_dotenv()

True

In [ ]:
import requests
import json

response = requests.get(
    url="https://openrouter.ai/api/v1/key",
    headers={"Authorization": f"Bearer {getenv("OPENROUTER_API_KEY")}"},
)
print(json.dumps(response.json(), indent=2))

In [2]:
db = SQLDatabase.from_uri(getenv("SUPABASE_URI"))
extractor = DatabaseSchemaExtractor(db)

schema_data = extractor.get_schemas()

print(f"Извлечено таблиц: {len(schema_data)}")

Извлечено таблиц: 9


In [ ]:
db._inspector.get_ta

{'text': 'This dataset has information about the customer and its location. Use it to identify unique customers in the orders dataset and to find the orders delivery location.  At our system each order is assigned to a unique customer_id. This means that the same customer will get different ids for different orders. The purpose of having a customer_unique_id on the dataset is to allow you to identify customers that made repurchases at the store. Otherwise you would find that each order had a different customer associated with.'}

In [7]:
chunks = extractor.split_schemas_to_chunks(schema_data, chunk_size=600)
print(f"Чанков: {len(chunks)}")

with open("chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=4)

Чанков: 42


In [7]:
builder = SchemaDocumentBuilder()
documents = builder.create_documents(schema_data)
print(f"Создано документов: {len(documents)}")

with open("docs.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=4)

Создано документов: 9


In [28]:
embeddings = OllamaEmbeddings(
    model="dengcao/Qwen3-Embedding-0.6B:Q8_0", base_url="http://localhost:11434"
)

In [ ]:
table_name = "schema_embeddings_qwen06b"

supabase_client = create_client(
    supabase_url=getenv("SUPABASE_URL"), supabase_key=getenv("SUPABASE_KEY")
)


vector_store = SupabaseVectorStore(
    client=supabase_client,
    embedding=embeddings,
    table_name=table_name,
    query_name="match_documents",
)


print(f"✅ Vector store setup completed for table: {table_name}")

✅ Vector store setup completed for table: schema_embeddings_qwen06b


In [30]:
vector_store.add_documents(documents)

['9b2b1016-3e86-4b21-b59a-bc48cd6d9648',
 '141de0f4-411c-4a19-92ce-cd85d4956706',
 'a05543ba-8f0e-41f3-8549-d27068269887',
 'a36f654c-1d6e-4213-940a-ed73fe2bfaf4',
 '472971ab-521f-4096-bad3-9b372a338568',
 'fe7bdf36-fad3-47a2-a484-7097b2c30cb8',
 '260ffc2f-b636-4091-abe4-316657307cd4',
 '43cf2ad4-5bd3-4e96-afe1-435e10b4622d',
 'b604520e-7a09-4628-9128-5b019c6a1002']

In [ ]:
# Фильтр — словарь (аналог JS-объекта)
filter = {"source": "https://example.com"}

# Поиск топ-2 документов по запросу "biology"
# Поиск с возвратом документов и их score
similarity_search_with_score_results = (
    vector_store.similarity_search_with_relevance_scores(
        "местоположение покупателей", k=5
    )
)

# Вывод результатов
for doc, score in similarity_search_with_score_results:
    print(f"* [SIM={score:.3f}] [{doc.metadata.get('table_name')}]\n")

* [SIM=0.445] [geolocation]

* [SIM=0.405] [customers]

* [SIM=0.377] [sellers]

* [SIM=0.374] [category_name_translation]

* [SIM=0.360] [products]



In [147]:
MODEL_NAME = "mistralai/mistral-small-3.1-24b-instruct"
db = SQLDatabase.from_uri(getenv("SUPABASE_URI"))
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 6})

llm = ChatOpenAI(
    model=MODEL_NAME,
    api_key=getenv("OPENROUTER_API_KEY"),
    base_url=getenv("OPENROUTER_API_URL"),
    temperature=0,
)

d:\Repos\AI Agent\.venv\Lib\site-packages\langchain_community\utilities\sql_database.py:134: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(


In [ ]:
query = "order_id"

docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.metadata.get("table_name"))

category_name_translation
customers
geolocation
products
sellers
orders


In [117]:
# 2️⃣ Переписываем запрос для поиска
query_rewrite_prompt = PromptTemplate(
    input_variables=["user_query"],
    template="""
        Ты выполняешь функцию генерации поискового запроса для RAG поиска в базе данных.
        Из исходного вопроса нужно сформировать **короткую строку ключевых слов**, подходящую для поиска по содержимому таблиц и колонок.
        Не добавляй объяснений, описаний, таблиц и колонок — только ключевые слова.
        Сделай запрос максимально коротким и ёмким.

        Вопрос: {user_query}
        Поисковый запрос:
        """,
)
query_rewriter = query_rewrite_prompt | llm

In [118]:
# 3️⃣ Пользовательский запрос
user_query = (
    "С какими внешними ключами и как связаны столбцы таблицы orders? Ответь на русском"
)

In [119]:
# 4️⃣ Переписываем и ищем документы
rag_query_result = query_rewriter.invoke({"user_query": user_query})
rag_query = rag_query_result.content  # получить строку из объекта AIMessage
docs = retriever.get_relevant_documents(rag_query)

In [112]:
# 5️⃣ Собираем контекст
context_text = "\n\n".join([doc.page_content for doc in docs])

In [ ]:
# 6️⃣ Формируем ответ LLM с учётом контекста
answer_prompt = PromptTemplate(
    input_variables=["question", "context"],
    template="Используй контекст для ответа на вопрос.\n\nКонтекст:\n{context}\n\nВопрос:\n{question}\n\nОтветь подробно на русском.",
)
chain = LLMChain(llm=llm, prompt=answer_prompt)
answer = chain.run({"question": user_query, "context": context_text})

print("Ответ:\n", answer)